In [9]:
import _pickle as cPickle
import numpy as np
import h5py
from operator import itemgetter
import copy
import random
from tqdm import tqdm
from collections import Counter, defaultdict

This notebook assumes you have a fasta of sequences you want to cluster, and you have your own way of obtaining all-vs.-all sequence identity between sequences in the fasta.

Once you have the fasta and the all-vs.-all sequence identities pre-computed (here it is provided in the form of a 2-d numpy array)

## Step 1: Create the sequence to index mapping object

In [ ]:
toy_fa = "../../1_Computing_Similarity/example_input/ex3.fa"

In [ ]:
def get_seq_keys(path):
    with open(path, "r") as file:
        s1 = file.read()
    seq_keys = [i.split("\n")[0] for i in s1.split(">")[1:]]
    return seq_keys

seq_keys = sorted(get_seq_keys(toy_fa))
key_to_idx = defaultdict(int)
key_to_idx['placeholder'] = 0
for ni, i in enumerate(seq_keys):
    key_to_idx[i] = ni+1

with open("../example_intermediate/ex3_key_to_idx.pickle", 'wb') as file:
    cPickle.dump(key_to_idx, file)

# Step 2: Format identity matrix from ssearch36 formatted output

In [ ]:
toy_fa = "../../1_Computing_Similarity/example_input/ex3.fa"

In [43]:
def get_seq_keys(path):
    with open(path, "r") as file:
        s1 = file.read()
    seq_keys = [i.split("\n")[0] for i in s1.split(">")[1:]]
    return seq_keys

seq_keys = sorted(get_seq_keys(toy_fa))

In [ ]:
with open('../../1_Computing_Similarity/example_output/ex3_formatted', 'r') as file:
    content = file.read()
content = content.split("\n")

In [45]:
ident_mat = np.zeros((len(seq_keys), len(seq_keys)))

for i in tqdm(content):
    if len(i) > 0:
        row = i.split("\t")
        id1 = row[0]
        id2 = row[1]
        identity = float(row[2])
        min_len = np.minimum(int(row[3]), int(row[4]))
        aln_len = int(row[5])

        # custom coverage control
        coverage = min_len/aln_len
        if coverage > 0.25:
            ident_mat[seq_keys.index(id1), seq_keys.index(id2)] = identity
        else:
            ident_mat[seq_keys.index(id1), seq_keys.index(id2)] = 0


100%|██████████| 1000001/1000001 [00:16<00:00, 60686.75it/s]


In [ ]:
np.save('../example_input/ex3_pairwise_ident.npy', ident_mat)

# Step 3: Create the h5 object which stores and retrieves the pairwise identities

In [ ]:
# It is assumed you have pre-computed all pairwise distances / sequence similarity
# It is assumed this array is also ordered according to the order of `seq_keys` above
# i.e. identity of `seq_keys[0]` against `seq_keys[1]` can be found at `loaded_array[0, 1]`
# Here, an example array of size `len(seq_keys) x len(seq_keys)` is provided where each 
# entry is the identity of a pair of sequences.
loaded_array = np.load('../example_intermediate/ex3_pairwise_ident.npy')

In [ ]:
import h5py
import numpy as np

# Create an HDF5 file
# The actual H5 data structure will reserve the first row and first column as a 
# place-holder for any non-existing keys, which it will then return 0. i.e. the
# H5 structure will be of size `(len(seq_keys)+1) x (len(seq_keys)+1)`
with h5py.File('../example_intermediate/ex3.h5', 'w') as hdf:
    hdf.create_dataset('0', data=np.zeros(loaded_array.shape[0] + 1, dtype=float))  # index 0 is reserved for placeholder
    for ni, i in enumerate(tqdm(seq_keys)):
        ident_list = loaded_array[ni]   # identities of sequence i against all of seq_keys
        ident_list = np.concatenate([np.array([0.]), ident_list])   # index 0 is reserved for placeholder
        hdf.create_dataset(str(key_to_idx[i]), data=ident_list)


100%|██████████| 1000/1000 [00:00<00:00, 8614.55it/s]
